# AzureML experiment

### Initialise workspace
Workspace.from_config() creates a workspace object from the details stored in config.json.

In [8]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: prroy-odsc-europe
Azure region: uksouth
Resource group: prroy-rg


### Create a remote compute target
You will need to create a compute target to execute your training script on. In this tutorial, you create an Azure Batch AI cluster as your training compute resource. This code creates a cluster for you if it does not already exist in your workspace.<br>
Creation of the cluster takes approximately 5 minutes. If the cluster is already in your workspace this code will skip the cluster creation process.

In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=6)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-11-15T18:05:01.167000+00:00', 'errors': None, 'creationTime': '2019-11-15T18:04:53.714253+00:00', 'modifiedTime': '2019-11-15T18:05:09.519240+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


### Create a project directory

In [10]:
import os

project_folder = './pytorch-exp'
os.makedirs(project_folder, exist_ok=True)

### Training file

In [11]:
%%writefile train.py
import argparse
import os
import numpy as np
import time
import torch
import torchvision
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import (
    Variable,
)
from torch.nn import (
    init,
)
from torchvision import (
    datasets, 
    transforms,
    models,
    utils,
)

model_name = 'resnet18'
num_workers = 2
num_epochs = 2 
batch_size = 32
learning_rate = 0.01
momentum = 0.9
weight_decay = 1e-4
dropout_p = 0.4
decay_rate = 0.9999
max_grad_norm = 5.0
log_interval = 1
num_classes = 8

# reproduceability
seed = 42
torch.manual_seed(seed)

# Fetch the dataset(in rar format), unrar it after install unrar. 
os.system('wget https://www.rarlab.com/rar/rarlinux-x64-5.6.0.tar.gz')
os.system('tar -zxvf rarlinux-x64-5.6.0.tar.gz')
os.system('./rar/unrar')
os.system(
'wget http://vision.stanford.edu/lijiali/event_dataset/event_dataset.rar')
os.system('./rar/unrar x event_dataset.rar')

# Load the data, split it among train, test and validation set after applying a series of transforms.
image_folder = 'event_img/'
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.47637546, 0.485785  , 0.4522678 ], [0.24692202, 0.24377407, 0.2667196 ])
    ])
data = datasets.ImageFolder(root=image_folder, transform=data_transforms)
class_names = data.classes
train_len, val_len = int(len(data) * 0.75), int(len(data) * 0.2)
test_len = len(data) - train_len - val_len
train_set, val_set, test_set = torch.utils.data.random_split(data, [train_len, val_len, test_len])
loader = {
    'train': torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'test': torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'val': torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
}

# Pick one of the pre-trained models, replace its final layer setting its output to the number of classes.
model = models.__dict__[model_name](pretrained=True) # Set false to train from scratch
# Alter the final layer
final_layer_input = model.fc.in_features
# nn.Linear a linear transformation to the incoming data: y = x A^T + b
model.fc = nn.Linear(final_layer_input, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=learning_rate, 
                      momentum=momentum, 
                      weight_decay=weight_decay,
                     )


def process_batch(inputs, targets, model, criterion, optimizer, is_training):
    """
    Process a minibatch for loss and accuracy.
    """
    
    # Convert tensors to Variables (for autograd)
    if is_training:
        X_batch = Variable(inputs, requires_grad=False)
    else:
        X_batch = Variable(inputs, volatile=True, requires_grad=False)
    y_batch = Variable(targets.long(), requires_grad=False)

    # Forward pass
    scores = model(X_batch) # logits

    # Loss
    loss = criterion(scores, y_batch)
    
    # Accuracy
    score, predicted = torch.max(scores, 1)
    accuracy = (y_batch.data == predicted.data).sum() / float(len(y_batch))
    
    if is_training:

        # In PyTorch, we need to set the gradients to zero before starting to
        # do backpropragation because PyTorch accumulates the gradients on subsequent
        # backward passes.
        optimizer.zero_grad()
        loss.backward()
        
        # Clip the gradient norms
        nn.utils.clip_grad_norm(model.parameters(), max_grad_norm)

        # Update params
        optimizer.step()

    return loss, accuracy


def train(model, criterion, optimizer, train_loader, test_loader, 
          num_epochs, batch_size, log_interval, learning_rate,
          dropout_p, decay_rate, max_grad_norm):
    """
    Training the model.
    """
    
    # Metrics
    train_loss, train_acc = [], []
    test_loss, test_acc = [], []

    # Training
    for num_train_epoch in range(num_epochs):
        # Timer
        start = time.time()

        # Decay learning rate
        learning_rate = learning_rate * (decay_rate ** (num_train_epoch // 1.0))
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate

        # Metrics
        train_batch_loss, train_batch_accuracy = 0.0, 0.0

        for train_batch_num, (inputs, target) in enumerate(train_loader):
            # Get metrics
            model.train()
            loss, accuracy = process_batch(
                inputs, target, model, criterion, optimizer, model.training)
            
            # Add to batch scalars
            train_batch_loss += loss.data.item() / float(len(inputs))
            train_batch_accuracy += accuracy
            
        # Add to global metrics
        train_loss.append(train_batch_loss / float(train_batch_num+1))
        train_acc.append(train_batch_accuracy / float(train_batch_num+1))

        # Testing
        model.eval()
        for num_test_epoch in range(1):
            # Metrics
            test_batch_loss, test_batch_accuracy = 0.0, 0.0

            for test_batch_num, (inputs, target) in enumerate(test_loader):
                # Get metrics
                model.eval()
                loss, accuracy = process_batch(
                    inputs, target, model, criterion, optimizer, model.training)
                # Add to batch scalars
                test_batch_loss += loss.data.item() / float(len(inputs))
                test_batch_accuracy += accuracy

            # Add to global metrics
            test_loss.append(test_batch_loss / float(test_batch_num+1))
            test_acc.append(test_batch_accuracy / float(test_batch_num+1))
                

            verbose_condition = ((num_train_epoch == 0) or (num_train_epoch % log_interval == 0) 
                                 or (num_train_epoch == num_epochs-1))

            # Verbose
            if verbose_condition:
                time_remain = (time.time() - start) * (num_epochs - (num_train_epoch + 1))
                minutes = time_remain // 60
                seconds = time_remain - minutes * 60
                print(f'TIME REMAINING: {minutes:.0f}m {seconds:.0f}s')
                print(f'[EPOCH]: {num_train_epoch},'
                      f'[TRAIN LOSS]: {train_batch_loss / float(train_batch_num+1):.6f},'
                      f'[TRAIN ACC]: {train_batch_accuracy / float(train_batch_num+1):.3f},'
                      f'[VAL LOSS]: {test_batch_loss / float(test_batch_num+1):.6f},'
                      f'[VAL ACC]: {test_batch_accuracy / float(test_batch_num+1):.3f}')
    return model


model = train(model, criterion, optimizer, loader['train'], loader['val'], 
              num_epochs, batch_size, log_interval, learning_rate,
              dropout_p, decay_rate, max_grad_norm)

parser = argparse.ArgumentParser(
    description='PyTorch Sports Image Classification')
parser.add_argument('--output-dir', type=str, default='outputs')
args = parser.parse_args()
output_dir = args.output_dir
os.makedirs(output_dir, exist_ok=True)
dummy_input = torch.randn(1, 3, 224, 224)
model_path = os.path.join(output_dir, 'sports_classification-1.onnx')
torch.onnx.export(model, dummy_input, model_path)

Overwriting train.py


In [12]:
%%bash
mv train.py pytorch-exp

### Create an experiment
Create an Experiment to track all the runs in your workspace for this transfer learning PyTorch tutorial.

In [13]:
from azureml.core import Experiment

experiment_name = 'pytorch1-sports'
experiment = Experiment(ws, name=experiment_name)

### Create a PyTorch estimator
The AML SDK's PyTorch estimator enables you to easily submit PyTorch training jobs for both single-node and distributed runs. For more information on the PyTorch estimator, refer here(https://docs.microsoft.com/en-gb/azure/machine-learning/service/how-to-train-pytorch). The following code will define a single-node PyTorch job.

In [14]:
from azureml.train.dnn import PyTorch

estimator = PyTorch(source_directory=project_folder, 
                    script_params={'--output-dir': './outputs'},
                    compute_target=compute_target,
                    entry_script='train.py',
                    use_gpu=False)

# upgrade to PyTorch 1.0 Preview, which has better support for ONNX
estimator.conda_dependencies.remove_conda_package('pytorch=0.4.0')
estimator.conda_dependencies.add_conda_package('pytorch-nightly')
estimator.conda_dependencies.add_channel('pytorch')

WARNING - framework_version is not specified, defaulting to version 1.2.


The script_params parameter is a dictionary containing the command-line arguments to your training script entry_script. Please note the following:
We specified the output directory as ./outputs. The outputs directory is specially treated by AML in that all the content in this directory gets uploaded to your workspace as part of your run history. The files written to this directory are therefore accessible even once your remote run is over. In this tutorial, we will save our trained model to this output directory.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [15]:
run = experiment.submit(estimator)
print(run.get_details())

{'runId': 'pytorch1-sports_1573841553_c14dd35b', 'target': 'cpu-cluster', 'status': 'Queued', 'properties': {'_azureml.ComputeTargetType': 'batchai', 'ContentSnapshotId': 'c90762db-86e5-41e1-aca8-48f5204b94ec', 'azureml.git.repository_uri': 'https://github.com/prabhat00155/onnx-odsc-tutorial.git', 'mlflow.source.git.repoURL': 'https://github.com/prabhat00155/onnx-odsc-tutorial.git', 'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': 'd9f7fb4d41060734fff32a9134cd3b4ab64fd5b3', 'mlflow.source.git.commit': 'd9f7fb4d41060734fff32a9134cd3b4ab64fd5b3', 'azureml.git.dirty': 'True', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'runDefinition': {'script': 'train.py', 'arguments': ['--output-dir', './outputs'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'cpu-cluster', 'dataReferences': {}, 'data': {}, 'jobName': None, 'max

### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [16]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Alternatively, you can block until the script has completed training before running more code.

In [17]:
%%time
run.wait_for_completion(show_output=True)

RunId: pytorch1-sports_1573841553_c14dd35b
Web View: https://mlworkspace.azure.ai/portal/subscriptions/00c06639-6ee4-454e-8058-8d8b1703bd87/resourceGroups/prroy-rg/providers/Microsoft.MachineLearningServices/workspaces/prroy-odsc-europe/experiments/pytorch1-sports/runs/pytorch1-sports_1573841553_c14dd35b

Streaming azureml-logs/55_azureml-execution-tvmps_6d9cac5fb0a1a257ab136b3911b97a29239d1588b2d56b3e4bd12be098df8aed_d.txt

2019-11-15T18:15:19Z Starting output-watcher...
Login Succeeded
1.2-cpu: Pulling from pytorch
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling fs layer
a6c378d11cbf: Pulling fs layer
6cc007ad9140: Pulling fs layer
6c1698a608f3: Pulling fs layer
84cbe4879fa7: Pulling fs layer
b0a763e8ee03: Waiting
11917a028ca4: Waiting
a6c378d11cbf: Waiting
6cc007ad9140: Waiting
6c1698a608f3: Wai

  1000K .......... .......... .......... .......... ..........  0% 2.45M 23m47s
  1050K .......... .......... .......... .......... ..........  0%  412K 23m35s
  1100K .......... .......... .......... .......... ..........  0% 2.54M 22m42s
  1150K .......... .......... .......... .......... ..........  0%  413K 22m33s
  1200K .......... .......... .......... .......... ..........  0% 2.58M 21m47s
  1250K .......... .......... .......... .......... ..........  0%  413K 21m41s
  1300K .......... .......... .......... .......... ..........  0% 22.8M 20m54s
  1350K .......... .......... .......... .......... ..........  0% 2.74M 20m15s
  1400K .......... .......... .......... .......... ..........  0%  412K 20m13s
  1450K .......... .......... .......... .......... ..........  0% 2.68M 19m39s
  1500K .......... .......... .......... .......... ..........  0% 8.94M 19m2s
  1550K .......... .......... .......... .......... ..........  0%  425K 19m2s
  1600K .......... .......... .......... .

 24500K .......... .......... .......... .......... ..........  5% 17.1M 2m49s
 24550K .......... .......... .......... .......... ..........  5% 19.2M 2m49s
 24600K .......... .......... .......... .......... ..........  5% 21.0M 2m49s
 24650K .......... .......... .......... .......... ..........  5% 22.5M 2m48s
 24700K .......... .......... .......... .......... ..........  5% 23.7M 2m48s
 24750K .......... .......... .......... .......... ..........  5% 21.4M 2m48s
 24800K .......... .......... .......... .......... ..........  5% 24.4M 2m47s
 24850K .......... .......... .......... .......... ..........  5% 24.5M 2m47s
 24900K .......... .......... .......... .......... ..........  5% 24.4M 2m47s
 24950K .......... .......... .......... .......... ..........  5% 17.8M 2m46s
 25000K .......... .......... .......... .......... ..........  5% 22.1M 2m46s
 25050K .......... .......... .......... .......... ..........  5% 22.2M 2m46s
 25100K .......... .......... .......... .......... 

 58600K .......... .......... .......... .......... .......... 12% 21.2M 87s
 58650K .......... .......... .......... .......... .......... 12% 17.0M 87s
 58700K .......... .......... .......... .......... .......... 12% 18.3M 87s
 58750K .......... .......... .......... .......... .......... 12% 21.1M 87s
 58800K .......... .......... .......... .......... .......... 12% 24.8M 87s
 58850K .......... .......... .......... .......... .......... 12% 21.4M 87s
 58900K .......... .......... .......... .......... .......... 12% 16.2M 87s
 58950K .......... .......... .......... .......... .......... 12% 19.4M 87s
 59000K .......... .......... .......... .......... .......... 12% 22.0M 87s
 59050K .......... .......... .......... .......... .......... 12% 20.6M 87s
 59100K .......... .......... .......... .......... .......... 12% 18.2M 87s
 59150K .......... .......... .......... .......... .......... 12% 22.9M 87s
 59200K .......... .......... .......... .......... .......... 12% 21.0M 87s

 90850K .......... .......... .......... .......... .......... 18% 15.4M 65s
 90900K .......... .......... .......... .......... .......... 18% 15.8M 65s
 90950K .......... .......... .......... .......... .......... 18% 13.9M 65s
 91000K .......... .......... .......... .......... .......... 18% 7.17M 65s
 91050K .......... .......... .......... .......... .......... 18% 13.9M 65s
 91100K .......... .......... .......... .......... .......... 18% 20.2M 65s
 91150K .......... .......... .......... .......... .......... 18% 10.6M 65s
 91200K .......... .......... .......... .......... .......... 18% 7.66M 65s
 91250K .......... .......... .......... .......... .......... 18% 22.2M 65s
 91300K .......... .......... .......... .......... .......... 18% 13.5M 65s
 91350K .......... .......... .......... .......... .......... 18% 20.4M 65s
 91400K .......... .......... .......... .......... .......... 18% 6.14M 65s
 91450K .......... .......... .......... .......... .......... 18% 6.77M 65s

...

Extracting  event_img/snowboarding/Easy_Mid_snowboarding_16.jpg           93%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_160.jpg          93%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_162.jpg          93%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_164.jpg          93%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_168.jpg          93% 94%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_17.jpg           94%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_186.jpg          94%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_187.jpg          94%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_190.jpg          94%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_191.jpg          94%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_192.jpg        

Extracting  event_img/snowboarding/Easy_Mid_snowboarding_349.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_350.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_351.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_352.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_353.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_355.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_357.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_358.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_359.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_361.jpg          97%  OK 
Extracting  event_img/snowboarding/Easy_Mid_snowboarding_364.jpg          97

51.9%
52.0%
52.0%
52.0%
52.0%
52.0%
52.0%
52.1%
52.1%
52.1%
52.1%
52.1%
52.1%
52.2%
52.2%
52.2%
52.2%
52.2%
52.3%
52.3%
52.3%
52.3%
52.3%
52.3%
52.4%
52.4%
52.4%
52.4%
52.4%
52.4%
52.5%
52.5%
52.5%
52.5%
52.5%
52.6%
52.6%
52.6%
52.6%
52.6%
52.6%
52.7%
52.7%
52.7%
52.7%
52.7%
52.7%
52.8%
52.8%
52.8%
52.8%
52.8%
52.8%
52.9%
52.9%
52.9%
52.9%
52.9%
53.0%
53.0%
53.0%
53.0%
53.0%
53.0%
53.1%
53.1%
53.1%
53.1%
53.1%
53.1%
53.2%
53.2%
53.2%
53.2%
53.2%
53.3%
53.3%
53.3%
53.3%
53.3%
53.3%
53.4%
53.4%
53.4%
53.4%
53.4%
53.4%
53.5%
53.5%
53.5%
53.5%
53.5%
53.5%
53.6%
53.6%
53.6%
53.6%
53.6%
53.7%
53.7%
53.7%
53.7%
53.7%
53.7%
53.8%
53.8%
53.8%
53.8%
53.8%
53.8%
53.9%
53.9%
53.9%
53.9%
53.9%
54.0%
54.0%
54.0%
54.0%
54.0%
54.0%
54.1%
54.1%
54.1%
54.1%
54.1%
54.1%
54.2%
54.2%
54.2%
54.2%
54.2%
54.2%
54.3%
54.3%
54.3%
54.3%
54.3%
54.4%
54.4%
54.4%
54.4%
54.4%
54.4%
54.5%
54.5%
54.5%
54.5%
54.5%
54.5%
54.6%
54.6%
54.6%
54.6%
54.6%
54.7%
54.7%
54.7%
54.7%
54.7%
54.7%
54.8%
54.8%
54.8%
54.8%
54.8%
54.8

train.py:113: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(model.parameters(), max_grad_norm)
train.py:91: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  X_batch = Variable(inputs, volatile=True, requires_grad=False)
TIME REMAINING: 3m 11s
[EPOCH]: 0,[TRAIN LOSS]: 0.021756,[TRAIN ACC]: 0.762,[VAL LOSS]: 0.010146,[VAL ACC]: 0.873
TIME REMAINING: 0m 0s
[EPOCH]: 1,[TRAIN LOSS]: 0.008792,[TRAIN ACC]: 0.911,[VAL LOSS]: 0.007482,[VAL ACC]: 0.928


The experiment completed successfully. Finalizing run...
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.0005910396575927734 seconds

Streaming azureml-logs/75_job_post-tvmps_6d9cac5fb0a1a257ab136b3911b97a29239d1588b2d56b3e4bd12be098df8aed_d.txt

Starting job release. Current time:2019-11-15T18:28:51.820742
Logging experiment finalizing status in history servic

{'runId': 'pytorch1-sports_1573841553_c14dd35b',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2019-11-15T18:15:16.675412Z',
 'endTimeUtc': '2019-11-15T18:29:21.733293Z',
 'properties': {'_azureml.ComputeTargetType': 'batchai',
  'ContentSnapshotId': 'c90762db-86e5-41e1-aca8-48f5204b94ec',
  'azureml.git.repository_uri': 'https://github.com/prabhat00155/onnx-odsc-tutorial.git',
  'mlflow.source.git.repoURL': 'https://github.com/prabhat00155/onnx-odsc-tutorial.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'd9f7fb4d41060734fff32a9134cd3b4ab64fd5b3',
  'mlflow.source.git.commit': 'd9f7fb4d41060734fff32a9134cd3b4ab64fd5b3',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'arguments': ['--output-dir', './outputs'],
  'sourceDirectoryDataStore': None,
 

### Download the model (optional)
Once the run completes, you can choose to download the ONNX model.

In [18]:
# list all the files from the run
run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_6d9cac5fb0a1a257ab136b3911b97a29239d1588b2d56b3e4bd12be098df8aed_d.txt',
 'azureml-logs/65_job_prep-tvmps_6d9cac5fb0a1a257ab136b3911b97a29239d1588b2d56b3e4bd12be098df8aed_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_6d9cac5fb0a1a257ab136b3911b97a29239d1588b2d56b3e4bd12be098df8aed_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/129_azureml.log',
 'logs/azureml/azureml.log',
 'outputs/sports_classification-1.onnx']

In [19]:
model_path = os.path.join('outputs', 'sports_classification-1.onnx')
run.download_file(model_path, output_file_path=model_path)

### Register the model
You can also register the model from your run to your workspace. The model_path parameter takes in the relative path on the remote VM to the model file in your outputs directory. You can then deploy this registered model as a web service through the AML SDK.

In [20]:
model = run.register_model(model_name='sports_classification-1', model_path=model_path)
print(model.name, model.id, model.version, sep = '\t')

sports_classification-1	sports_classification-1:2	2


#### Displaying your registered models (optional)
You can optionally list out all the models that you have registered in this workspace.

In [21]:
models = ws.models
for name, m in models.items():
    print("Name:", name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

Name: sports_classification-1 	Version: 2 	Description: None {}
Name: sports_classification-pretrained 	Version: 3 	Description: Sports image classification {'onnx': 'demo'}


## Deploying as a web service
Write scoring file
We are now going to deploy our ONNX model on Azure ML using the ONNX Runtime. We begin by writing a score.py file that will be invoked by the web service call. The init() function is called once when the container is started so we load the model using the ONNX Runtime into a global session object.

In [22]:
%%writefile score.py
import json
import time
import sys
import os
from PIL import Image
import requests
from io import BytesIO
from azureml.core.model import Model
import numpy as np
from onnxruntime import InferenceSession
from torchvision import transforms

def init():
    global session
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_onnx = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sports_classification-1.onnx')
    session = InferenceSession(model_onnx)

def preprocess(input_data_json):
    input_url = json.loads(input_data_json)['data'][0]
    # convert the image url into the tensor input
    data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.47637546, 0.485785  , 0.4522678 ], [0.24692202, 0.24377407, 0.2667196 ])
    ])  
    response = requests.get(input_url)
    image = Image.open(BytesIO(response.content))
    image = data_transforms(image)
    image = image.numpy().reshape((1, *image.shape))
    return image

def postprocess(result):
    class_names = [ 
        'RockClimbing',
        'badminton',
        'bocce',
        'croquet',
        'polo',
        'rowing',
        'sailing',
        'snowboarding'
    ]   
    return class_names[np.argmax(result[0])]

def run(input_data_json):
    try:
        start = time.time()   # start timer
        input_data = preprocess(input_data_json)
        input_name = session.get_inputs()[0].name  # get the id of the first input of the model   
        result = session.run(None, {input_name: input_data})
        end = time.time()     # stop timer
        return {"result": postprocess(result),
                "time": end - start}
    except Exception as e:
        result = str(e)
        return {"error": result}


Overwriting score.py


### Create inference configuration
First we create a YAML file that specifies which dependencies we would like to see in our container.

In [23]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=["numpy","onnxruntime","azureml-core", "Pillow", "torchvision"])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Then we setup the inference configuration

In [24]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="myenv.yml",
                                   extra_docker_file_steps = "Dockerfile")

### Deploy the model

In [25]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'web service for sports classification ONNX model')

The following cell will likely take a few minutes to run as well.

In [26]:
from azureml.core.model import Model
from random import randint

aci_service_name = f'onnx-sports{randint(0,100)}'
print("Service", aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Service onnx-sports88
Creating image
Running............................................................
Succeeded
Image creation operation finished for image onnx-sports88:1, operation "Succeeded"
Running...............................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In case the deployment fails, you can check the logs. Make sure to delete your aci_service before trying again.

In [27]:
if aci_service.state != 'Healthy':
    # run this command for debugging.
    print(aci_service.get_logs())
    aci_service.delete()

### Success!
If you've made it this far, you've deployed a working web service that does sports image classification using an ONNX model. You can get the URL for the webservice with the code below.

In [28]:
service_url = aci_service.scoring_uri
service_url

'http://c3aa0787-6c90-44bc-9eff-775ac9dd2693.uksouth.azurecontainer.io/score'

### Test the service
To submit sample data to the running service, use the following code.

In [31]:
from IPython.display import Image

image_name = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/DN_ice_boat--Ice_Nine--Lake_Sunapee_NH.jpg/220px-DN_ice_boat--Ice_Nine--Lake_Sunapee_NH.jpg'
Image(url= image_name)

In [30]:
import requests
import json
test_sample = json.dumps({'data': [
    image_name
]})
test_sample = bytes(test_sample,encoding = 'utf8')
headers = {'Content-Type':'application/json'}
resp = requests.post(service_url, test_sample, headers=headers)
print("prediction:", resp.text)

prediction: {"result": "sailing", "time": 0.5485150814056396}
